In [ ]:
from scipy.stats import gaussian_kde

import json

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [ ]:
with open('/Volumes/thesis-data/dtype_dict.json') as f:
    dtypes = json.load(f)

df = pd.read_csv('/Volumes/thesis-data/main.csv', dtype=dtypes,
                 parse_dates=['PERIOD', 'ADMDATE', 'DISCDATE'])

# Cost variation

Our objective is to better understand cost variation in our data, so let us focus our analysis on that for now. We begin by determining the components of costs with the highest variation.

- Coefficient of variation is a standardised measure
- Use this as a model for regression to understand what makes the variation

In [ ]:
costs = [
    'COST', 'NetCost', 'DRUG', 'ENDO', 'HCD',
    'IMG', 'IMG_OTH', 'MED', 'NCI', 'NID', 'OCLST', 'OPTH',
    'OTH', 'OTH_OTH', 'OUTP', 'OVH', 'PATH', 'PATH_OTH',
    'PHAR', 'PROS', 'RADTH', 'SECC', 'SPS', 'THER', 'WARD'
]

We collect the total of all cost variables in a spell, i.e. the total cost of each episode in every spell.

In [ ]:
sum_costs = df.groupby(['PATIENT_ID', 'SPELL_ID'])[costs].sum().reset_index() \
                                                         .rename({cost : f'{cost}_sum' for cost in costs}, axis=1)

trim_sum_costs = sum_costs[sum_costs['NetCost_sum'] <= 20000]

In [ ]:
just_costs = sum_costs.drop(['PATIENT_ID', 'SPELL_ID'], axis=1)

just_costs.var().round(2).sort_values(ascending=False)

## Net cost distribution

In [ ]:
def get_kde(df, col, steps=300):

    data = df[col]
    xs = np.linspace(data.min(), data.max(), steps)
    density = gaussian_kde(data)

    return xs, density

Taking this trim is not necessarily that useful. Sure, if you take a tail off you get a "nicer" distribution and people cost less on the whole. What would be better is to incorporate the carbon-copy stories and say these two very similar people are far apart (one in the tail, the other in the nose) but if you changed this thing about them you could move that person person in the tail up. Thus, saving potentially £1000s.

In [ ]:
fig, ax = plt.subplots(1, figsize=(12, 8))

for name, dataset in zip(['Full', 'Trimmed'], [sum_costs, trim_sum_costs]):
    xs, density = get_kde(dataset, 'NetCost_sum')
    ax.plot(xs, density(xs), label=name)
    ax.fill_between(xs, [0]*xs, density(xs), alpha=0.2)

ax.set_title('Gaussian KDE for net cost of a spell')
ax.set_xlim(0, 20000)
ax.legend(loc='best');

# Diabetes analysis

This is a known area of interest to the health board.

We begin by defining a new column in the dataset which combines the diabetes primary/secondary diagnosis columns. From here we can analyse the proportion of admissions that are diabetic and average length of stays of diabetic patients. This is done over time using a rolling 28-day mean and yearly resampling.

Note that we trim the dataset to only include episodes with admission dates from April 2013 onwards and discharge dates prior to May 2017; this is because not all of our data sources range over the same periods of time.

In [ ]:
time_constraint = (df['ADMDATE'] >= '2012-04-01') & (df['DISCDATE'] < '2017-03-01')

In [ ]:
diabetes_present = (df['DiabetesPrim'] == '1') | (df['DiabetesSec'] == '1')
df['Diabetes'] = np.where(diabetes_present, 1, 0)

## Diabetic admissions as a proportion of the total per day

Let's calculate the number of admissions for the whole dataset and for diabetic patients each day. This is equivalent to counting the number of unique spells every day.

- Reworking the length of stay, finding the proportion of 'resources' (cost, beds, etc) that are diabetic

In [ ]:
num_admissions = df[time_constraint].groupby(['ADMDATE']).SPELL_ID.nunique().reset_index() \
                                                         .rename({'SPELL_ID': 'nunique_spells'}, axis=1) \
                                                         .groupby('ADMDATE').nunique_spells.sum()

In [ ]:
diab_admissions = df[time_constraint].groupby(['ADMDATE', 'PATIENT_ID', 'SPELL_ID']).Diabetes.max().reset_index() \
                                                                                    .groupby(['ADMDATE']).Diabetes.sum()

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10), dpi=300)

proportions = diab_admissions / num_admissions
rolling = proportions.rolling(28, center=True)
mean = rolling.mean()
monthly = proportions.resample('BM').mean()
yearly = proportions.resample('BA-APR').mean()

# ax.plot(proportions, '.', alpha=0.1, label='input data')
# ax.plot(mean, '-', label='28 day rolling mean')
ax.plot(monthly, '.-', label='end of month average')
ax.plot(yearly, 'o', label='end of year average')

ax.set_title('Diabetic admissions as a proportion of total admissions')
ax.legend(loc='best');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 8))

ax.boxplot(proportions, notch=True, vert=False, showmeans=True);

## Average length of stay by admission date for diabetic patients

Here we contrast the average length of stay for patients in a single spell (i.e. the total length of their episodes) for diabetic patients, non-diabetic patients and the general population.

In [ ]:
diab_LOS = df[(df['Diabetes'] == 1) & time_constraint].groupby(['ADMDATE', 'PATIENT_ID', 'SPELL_ID']) \
                                                      .LOS.sum().reset_index().rename({'LOS': 'LOS_sum'}, axis=1) \
                                                      .groupby('ADMDATE').LOS_sum.mean()

In [ ]:
general_LOS = df[time_constraint].groupby(['Diabetes', 'ADMDATE', 'PATIENT_ID', 'SPELL_ID']) \
                                 .LOS.sum().reset_index().rename({'LOS': 'LOS_sum'}, axis=1) \
                                 .groupby(['Diabetes', 'ADMDATE']).LOS_sum.mean()

without_LOS = df[(df['Diabetes'] == 0) & time_constraint].groupby(['ADMDATE', 'PATIENT_ID', 'SPELL_ID']) \
                                                         .LOS.sum().reset_index().rename({'LOS': 'LOS_sum'}, axis=1) \
                                                         .groupby('ADMDATE').LOS_sum.mean()

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 10), dpi=300)

rolling = diab_LOS.rolling(28, center=True)
mean = rolling.mean()
monthly = diab_LOS.resample('BM').mean()
yearly = diab_LOS.resample('BA-APR').mean()

# ax.plot(diab_LOS, '.', alpha=0.25, label='input data')
# ax.plot(mean, '-', label='rolling mean')
ax.plot(monthly, '.-', label='end of month average')
ax.plot(yearly, 'o', label='end of year average')
ax.set_ylabel('Average length of stay [days]')
ax.set_title('The average length of stay for diabetic patients given their admission date')
ax.legend(loc='best');

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 8))

ax.boxplot([diab_LOS.values, without_LOS.values, general_LOS.values],
           vert=False, positions=[1, 2, 3], sym='.', showmeans=True, notch=True)
ax.set_xlabel('Average length of stay')
ax.set_yticks([1, 2, 3])
ax.set_yticklabels(['diabetic', 'non-diabetic', 'general']);

## True LOS

In [ ]:
df['TRUE_LOS'] = df['DISCDATE'] - df['ADMDATE']
df['TRUE_LOS'] = df.TRUE_LOS.dt.days

In [ ]:
true_los = df.groupby(['PATIENT_ID', 'SPELL_ID']).TRUE_LOS.sum()

In [ ]:
wrong_discdate = true_los.iloc[np.where(true_los < 0)].reset_index()
spell_ids = wrong_discdate.SPELL_ID

wrong_discdate

In [ ]:
for spell_id in spell_ids:

    dates = df[df['SPELL_ID'] == spell_id][['ADMDATE', 'DISCDATE', 'LOS']]
    idx = dates.index
    if dates['DISCDATE'].nunique() > 1:

        latest = dates['DISCDATE'].max()
        new_discdates = np.where(dates['DISCDATE'] <= latest, pd.to_datetime(latest), np.nan)

        df.loc[df['SPELL_ID'] == spell_id, 'DISCDATE'] = new_discdates

In [ ]:
dates

In [ ]:
df['DISCDATE'] = pd.to_datetime(df['DISCDATE'])

In [ ]:
df['TRUE_LOS'] = df['DISCDATE'] - df['ADMDATE']
df['TRUE_LOS'] = df.TRUE_LOS.dt.days

true_los = df.groupby(['PATIENT_ID', 'SPELL_ID']).TRUE_LOS.sum()

In [ ]:
wrong_discdate = true_los.iloc[np.where(true_los < 0)].reset_index()
spell_ids = wrong_discdate.SPELL_ID

wrong_discdate

In [ ]:
discharge_dates = df[df['SPELL_ID'] == 'M1001789573'][['PATIENT_ID', 'ADMDATE', 'DISCDATE', 'LOS']]
discharge_dates

In [ ]:
discharge_dates = df.loc[df['SPELL_ID'] == 'M1000711976'][['PATIENT_ID', 'ADMDATE', 'DISCDATE', 'LOS']]
# new_dates = np.where(
#     discharge_dates['DISCDATE'] <= discharge_dates['DISCDATE'].max(),
#     discharge_dates['DISCDATE'].max(), np.nan
# )

# discharge_dates['DISCDATE'] = new_dates
discharge_dates

In [ ]:
positive_LOS = true_los.iloc[np.where(true_los > 0)].reset_index()
spell_ids = positive_LOS.SPELL_ID

for spell_id in spell_ids:
    length = df[df['SPELL_ID'] == spell_id][['PATIENT_ID', 'SPELL_ID', 'ADMDATE', 'DISCDATE', 'LOS']]
    print(length)

In [ ]:
idxs